# OpenTelemetry with OpenAI API

This notebook demonstrates how to use OpenTelemetry to trace OpenAI API calls.

Learn more about OpenTelemetry ingestion using MaximAI at https://www.getmaxim.ai/docs/tracing/opentelemetry/ingesting-via-otlp

## Install required packages

In [ ]:
%%capture --no-stderr
%pip install openai
%pip install opentelemetry-api
%pip install opentelemetry-exporter-otlp
%pip install opentelemetry-exporter-otlp-proto-http
%pip install opentelemetry-sdk
%pip install python-dotenv

In [ ]:
import json
import os
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry import trace
from openai import OpenAI
from dotenv import load_dotenv

## Load Environment Variables

Rename `.env.example` to `.env` and fill in the following variables:
- `MAXIM_API_KEY` - Your Maxim API key
- `MAXIM_REPO_ID` - Your Maxim Log repository ID
- `OPENAI_API_KEY` - Your OpenAI API key

In [ ]:
# Load environment variables
load_dotenv()

# Get API keys and IDs from environment variables
maxim_api_key = os.getenv("MAXIM_API_KEY")
repo_id = os.getenv("MAXIM_REPO_ID")

# Check if environment variables are set
if not maxim_api_key or not repo_id:
    print("Warning: MAXIM_API_KEY or MAXIM_REPO_ID environment variables are not set.")
    raise ValueError("MAXIM_API_KEY or MAXIM_REPO_ID environment variables are not set.")

## Set up OpenTelemetry with Maxim OTLP ingest endpoint

In [ ]:
# Set up OpenTelemetry tracer provider
tracer_provider = trace_sdk.TracerProvider()

# Configure OTLP exporter
span_exporter = OTLPSpanExporter(
    endpoint="https://api.getmaxim.ai/v1/otel",
    headers={
        "x-maxim-api-key": maxim_api_key,
        "x-maxim-repo-id": repo_id,
    },
)

# Add span processors
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter))

# Set the global tracer provider
trace.set_tracer_provider(tracer_provider)

# Create a tracer
tracer = trace.get_tracer("maxim_test_tracer")

## Initialize OpenAI Client

In [ ]:
# Initialize OpenAI client
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    print("Warning: OPENAI_API_KEY environment variable is not set.")
    raise ValueError("OPENAI_API_KEY environment variable is not set.")

client = OpenAI()

## Make OpenAI API Call with OpenTelemetry Tracing

In [ ]:
with tracer.start_as_current_span("genai_chat_completion") as span:
    # Set GenAI span attributes
    span.set_attribute("gen_ai.system", "openai")
    span.set_attribute("gen_ai.request.model", "gpt-4o-mini")
    span.set_attribute("gen_ai.request.max_tokens", 400)

    system_message = (
        "You are a helpful assistant and you are going to answer in one sentence only."
    )
    user_message = "What is LLM Observability?"

    # System message event
    span.add_event(
        "gen_ai.system.message",
        attributes={
            "gen_ai.system": "openai",
            "content": system_message,
            "role": "system",
        },
    )

    # User message event
    span.add_event(
        "gen_ai.user.message",
        attributes={
            "gen_ai.system": "openai",
            "content": user_message,
            "role": "user",
        },
    )

    # Make the API call
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": system_message,
            },
            {
                "role": "user",
                "content": user_message,
            },
        ],
        model="gpt-4o-mini",
        max_tokens=400,
    )

    # Set response attributes
    span.set_attribute("gen_ai.response.id", chat_completion.id)
    span.set_attribute("gen_ai.response.model", chat_completion.model)
    span.set_attribute("gen_ai.usage.input_tokens", chat_completion.usage.prompt_tokens)
    span.set_attribute(
        "gen_ai.usage.output_tokens", chat_completion.usage.completion_tokens
    )

    # Assistant message event (optional - captures the response)
    span.add_event(
        "gen_ai.assistant.message",
        attributes={
            "gen_ai.system": "openai",
            "content": chat_completion.choices[0].message.content,
            "role": "assistant",
        },
    )

    # Choice events - one for each choice
    for choice in chat_completion.choices:
        choice_body = {
            "finish_reason": choice.finish_reason,
            "index": choice.index,
            "message": {
                "content": choice.message.content,
                "role": choice.message.role,
            },
        }

        # Only include tool_calls if present
        if choice.message.tool_calls:
            choice_body["message"]["tool_calls"] = [
                {
                    "id": tool_call.id,
                    "type": tool_call.type,
                    "function": {
                        "name": tool_call.function.name,
                        "arguments": tool_call.function.arguments,
                    },
                }
                for tool_call in choice.message.tool_calls
            ]

        span.add_event(
            "gen_ai.choice",
            attributes={
                "finish_reason": choice_body["finish_reason"],
                "index": choice_body["index"],
                "message": json.dumps(choice_body["message"]),
            },
        )

## Display the Response

In [ ]:
# Display the response
print("Response from OpenAI:")
print(chat_completion.choices[0].message.content)